# Initialization

In [1]:
from PIL import Image, ExifTags
from wand.image import Image as WandImage
import getexifdata, glob, os, json, io, shutil, logprogress, numpy

DIR_PLACES = "../s3/"
DIR_TEMPLATE = "../s3/_PlaceTemplate/"
DIR_GEODAT = "geojson/"
DIR_AUD = "aud/"
DIR_IMG_ORIG = "imgOrig/"
DIR_IMG_LG = "imgLg/"
DIR_IMG_THUMBNAIL = "imgSm/"
DIR_IMG_ERR = "imgErr/"

IMG_FORMAT = ".jpg"
AUD_FORMAT = ".mp3"

INPUT_JSON = "info_template.json"
OUTPUT_JSON = "info.json"
SMRY_JSON = "all_rivers.json"

SIZE_IMG_THUMBNAIL = 400

# likely prefixes for original images named by the os when pic taken
BLACKLIST_FLABEL_PREFIX = ["IMG", "MVIMG", "PANO"]

####################
###
### helper functions
###

# if dir exists, clear it
def init_dir(path):
    if os.path.isdir(path):
        shutil.rmtree(path)
    os.makedirs(path)

def strip_dir(fpath, dir):
    i = fpath.index(dir)+len(dir)
    return fpath[i:len(fpath)]

def get_fname(fpath):
    return fpath.split("/")[-1]

def get_flabel(fpath, ext):
    return get_fname(fpath).replace(ext, "")

def find_audio_for_img(label):
    for fpath in glob.glob(base_path + DIR_AUD + "*" + AUD_FORMAT):
        flabel = get_flabel(fpath, AUD_FORMAT)
        if flabel == label:
            return strip_dir(fpath, "/" + DIR_AUD)
    return None # no warning needed

def for_img_with_exif(fpath, fnExif, fnNoExif):
    im = get_im(fpath)
    edat = getexifdata.get_exif_data(im)
    lat, lng = getexifdata.get_lat_lon(edat)
    if lat == None or lng == None:
        if fnNoExif != None:
            fnNoExif(fpath, im)
        return None
    else:
        if fnExif != None:
            fnExif(fpath, im)
        return {"lat":lat, "lng":lng}

def get_im(fpath):
    try:
        return Image.open(fpath)
    except:
        return as_pil_image(fpath)

# convert HEIF images to PIL format
def as_pil_image(fpath):
    wand_im = WandImage(filename=fpath)
    img_buffer = numpy.asarray(bytearray(wand_im.make_blob(format='png')), dtype='uint8')
    bytesio = io.BytesIO(img_buffer)
    return Image.open(bytesio)

def get_gps_for_fpath(fpath):
    return for_img_with_exif(fpath, None, None)

def get_date_for_fpath(fpath):
    im = Image.open(fpath)
    edat = getexifdata.get_exif_data(im)
    if "DateTime" in edat:
        date = edat["DateTime"]
    elif "DateTimeOriginal" in edat:
        date = edat["DateTimeOriginal"]
    else:
        date = "(Date Unknown)"
    return date

try:
    to_unicode = unicode
except NameError:
    to_unicode = str    

def find_geodat():
    return [strip_dir(fpath, "/" + DIR_GEODAT) for fpath in glob.glob(base_path + DIR_GEODAT + "*.geojson")]

def reorient_img(fileName, height):
    # thanks storm_to : http://stackoverflow.com/questions/4228530/pil-thumbnail-is-rotating-my-image 
    fpath = base_path + DIR_IMG_LG + fileName
    image=Image.open(fpath)
    exif_raw = image._getexif()
    if (exif_raw):
        for orientation in ExifTags.TAGS.keys() : 
            if ExifTags.TAGS[orientation]=='Orientation' : break 
        exif=dict(exif_raw.items())

        try:
            if   exif[orientation] == 3 : 
                image=image.rotate(180, expand=True)
            elif exif[orientation] == 6 : 
                image=image.rotate(270, expand=True)
            elif exif[orientation] == 8 : 
                image=image.rotate(90, expand=True)
        except KeyError:
            False
            #print("No orientation EXIF data for: " + fileName)

    # thumnail
    r = float(height) / image.size[1]
    w = float(image.size[0]) * r
    image.thumbnail((w, height), Image.ANTIALIAS)
    image.save(base_path + DIR_IMG_THUMBNAIL + fileName)

####################
###
### sanity check
###

# sanity check: every audio file should have a matching img
def ensure_audio_img_match():
    # error if an audio file has no image. slow, but we should ensure this.
    for fpath in glob.glob(base_path + DIR_AUD + "*" + AUD_FORMAT):
        match = False
        for imgpath in glob.glob(base_path + DIR_IMG_LG + "*" + IMG_FORMAT):
            if get_flabel(imgpath, IMG_FORMAT) == get_flabel(fpath, AUD_FORMAT):
                match = True
        if not match:
            #raise FileNotFoundError("no image for audio file: " + fpath)
            return True

# ensure each final image file is the desired extension & has GPS info
def ensure_all_img_sanity():
    [ensure_img_sanity(fpath) for fpath in glob.glob(base_path + DIR_IMG_ORIG + "*")]

def ensure_img_sanity(fpath):
    print("ensure_img_sanity", fpath)
    def fnExif(fpath, im):
        extension = "." + fpath.split(".")[-1]
        im.save(base_path + DIR_IMG_LG + get_fname(fpath).replace(extension, IMG_FORMAT), exif=im.info["exif"])
    def fnNoExif(fpath, im):
        im.save(base_path + DIR_IMG_ERR + get_fname(fpath).replace(IMG_FORMAT, "") + "_WARNING! no gps data for image" + IMG_FORMAT)
    for_img_with_exif(fpath, fnExif, fnNoExif)
    
# only permit location labels that were most likely hand-named image filenames
def verify_flabel(flabel):
    # invalid label if filename begins with '201' (likely a date)
    try:
        flabel.index("201")
        return False
    except ValueError:
        False # ignore
    # invalid label if blacklisted
    for prefix in BLACKLIST_FLABEL_PREFIX:
        try:
            flabel.index(prefix)
            return False
        except ValueError:
            continue
    return True

# Run

In [2]:
# process every dir in Places/
placels = [x for x in logprogress.log_progress(glob.glob(DIR_PLACES + "*/")) if x != DIR_TEMPLATE]
all_places = {"places":[]}

# process each directory
for base_path in placels:

    print("processing place: " + base_path)
    
    # clear workspace
    init_dir(base_path + DIR_IMG_LG)
    init_dir(base_path + DIR_IMG_THUMBNAIL)
    init_dir(base_path + DIR_IMG_ERR)
    
    ensure_all_img_sanity()
    ensure_audio_img_match()

    # read json template
    with open(base_path + INPUT_JSON) as data_file:
        data = json.load(data_file)
        data["layers"] = find_geodat()

        # for everyimage...
        files = glob.glob(base_path + DIR_IMG_LG + "*" + IMG_FORMAT)
        for fpath in logprogress.log_progress(files):
            
            # compile location info
            flabel = get_flabel(fpath, IMG_FORMAT)
            loc = get_gps_for_fpath(fpath)
            
            if loc != None:
                fnam = get_fname(fpath)
                date = get_date_for_fpath(fpath)
                
                marker = {
                    "date": date,
                    "loc": loc,
                    "img": fnam,
                    "aud": find_audio_for_img(flabel)
                }
                
                if verify_flabel(flabel):
                    marker["label"] = flabel
                
                data["locations"].append(marker)

                # save web-friendly image (rotated & small)
                reorient_img(fnam, SIZE_IMG_THUMBNAIL)
        
        # Write JSON file
        # http://stackoverflow.com/questions/12309269/how-do-i-write-json-data-to-a-file-in-python
        with io.open(base_path + OUTPUT_JSON, 'w', encoding='utf8') as outfile:
            str_ = json.dumps(data,
                              indent=4, sort_keys=True,
                              separators=(',', ':'), ensure_ascii=False)
            outfile.write(to_unicode(str_))
    
    # add to summary json
    dirName = base_path.split("/")[-2]
    dispName = dirName.replace("_", " ")
    
    all_places["places"].append({
        "id": dirName,
        "disp": dispName
    })

# write summary json: all place names
with io.open(DIR_PLACES + SMRY_JSON, 'w', encoding='utf8') as outfile:
    str_ = json.dumps(all_places,
                      indent=4, sort_keys=True,
                      separators=(',', ':'), ensure_ascii=False)
    outfile.write(to_unicode(str_))
            
print("done.")

processing place: ../s3/Bay_Area/
ensure_img_sanity ../s3/Bay_Area/imgOrig/2017-03-19 13.10.46.jpg
ensure_img_sanity ../s3/Bay_Area/imgOrig/2017-03-19 08.52.23.jpg
ensure_img_sanity ../s3/Bay_Area/imgOrig/2017-02-25 11.17.49 HDR.jpg
ensure_img_sanity ../s3/Bay_Area/imgOrig/2017-03-19 10.21.54 HDR.jpg
ensure_img_sanity ../s3/Bay_Area/imgOrig/2017-02-11 11.15.50 HDR.jpg
ensure_img_sanity ../s3/Bay_Area/imgOrig/2017-03-19 12.45.55 HDR.jpg
ensure_img_sanity ../s3/Bay_Area/imgOrig/2017-03-19 11.02.01.jpg
ensure_img_sanity ../s3/Bay_Area/imgOrig/2017-03-19 09.51.01.jpg
ensure_img_sanity ../s3/Bay_Area/imgOrig/2017-02-11 13.42.02 HDR.jpg
ensure_img_sanity ../s3/Bay_Area/imgOrig/2017-03-19 10.14.49.jpg
ensure_img_sanity ../s3/Bay_Area/imgOrig/2017-02-11 13.44.55.jpg
ensure_img_sanity ../s3/Bay_Area/imgOrig/2017-02-11 13.42.10 HDR.jpg
ensure_img_sanity ../s3/Bay_Area/imgOrig/AA8B29EA-BB44-457E-AF42-6327FED80850.heic
ensure_img_sanity ../s3/Bay_Area/imgOrig/2017-01-29 10.34.22.jpg
ensure_img_san

processing place: ../s3/Costa_Rica_2023/
ensure_img_sanity ../s3/Costa_Rica_2023/imgOrig/IMG_7498.jpeg
ensure_img_sanity ../s3/Costa_Rica_2023/imgOrig/IMG_7135.jpeg
ensure_img_sanity ../s3/Costa_Rica_2023/imgOrig/IMG_7375.jpeg
ensure_img_sanity ../s3/Costa_Rica_2023/imgOrig/IMG_7355.jpeg
ensure_img_sanity ../s3/Costa_Rica_2023/imgOrig/IMG_7139.jpeg
ensure_img_sanity ../s3/Costa_Rica_2023/imgOrig/IMG_7568.jpeg
ensure_img_sanity ../s3/Costa_Rica_2023/imgOrig/IMG_7304.jpeg
ensure_img_sanity ../s3/Costa_Rica_2023/imgOrig/IMG_7040.jpeg
ensure_img_sanity ../s3/Costa_Rica_2023/imgOrig/IMG_7105.jpeg
ensure_img_sanity ../s3/Costa_Rica_2023/imgOrig/IMG_7294.jpeg
ensure_img_sanity ../s3/Costa_Rica_2023/imgOrig/IMG_7060.jpeg
ensure_img_sanity ../s3/Costa_Rica_2023/imgOrig/IMG_7037.jpeg
ensure_img_sanity ../s3/Costa_Rica_2023/imgOrig/IMG_7148.jpeg
ensure_img_sanity ../s3/Costa_Rica_2023/imgOrig/IMG_7299.jpeg
ensure_img_sanity ../s3/Costa_Rica_2023/imgOrig/IMG_7036.jpeg
ensure_img_sanity ../s3/Costa

processing place: ../s3/San_Joaquin_River/


processing place: ../s3/Feather_River/
ensure_img_sanity ../s3/Feather_River/imgOrig/2017-07-23 14.17.34.jpg
ensure_img_sanity ../s3/Feather_River/imgOrig/2017-07-23 13.42.53.jpg
ensure_img_sanity ../s3/Feather_River/imgOrig/2017-06-24 12.52.20-1.jpg
ensure_img_sanity ../s3/Feather_River/imgOrig/2017-07-23 08.31.45 HDR.jpg
ensure_img_sanity ../s3/Feather_River/imgOrig/2017-07-23 11.16.33 HDR.jpg
ensure_img_sanity ../s3/Feather_River/imgOrig/2017-07-22 11.40.34 HDR.jpg
ensure_img_sanity ../s3/Feather_River/imgOrig/2017-07-23 14.12.16 HDR.jpg
ensure_img_sanity ../s3/Feather_River/imgOrig/2017-06-25 14.19.40.jpg
ensure_img_sanity ../s3/Feather_River/imgOrig/2017-06-25 14.07.43 HDR.jpg
ensure_img_sanity ../s3/Feather_River/imgOrig/2017-07-23 13.12.17 HDR.jpg
ensure_img_sanity ../s3/Feather_River/imgOrig/2017-07-22 12.21.30.jpg
ensure_img_sanity ../s3/Feather_River/imgOrig/2017-06-25 14.00.27-1.jpg
ensure_img_sanity ../s3/Feather_River/imgOrig/2017-07-23 11.35.12.jpg
ensure_img_sanity ../s3

processing place: ../s3/Russian_River/
ensure_img_sanity ../s3/Russian_River/imgOrig/IMG_4299.jpg
ensure_img_sanity ../s3/Russian_River/imgOrig/IMG_4514.jpg
ensure_img_sanity ../s3/Russian_River/imgOrig/IMG_4500.jpg
ensure_img_sanity ../s3/Russian_River/imgOrig/IMG_4058.jpg
ensure_img_sanity ../s3/Russian_River/imgOrig/IMG_4104.jpg
ensure_img_sanity ../s3/Russian_River/imgOrig/IMG_4460.jpg
ensure_img_sanity ../s3/Russian_River/imgOrig/IMG_4313.jpg
ensure_img_sanity ../s3/Russian_River/imgOrig/IMG_4529.jpg
ensure_img_sanity ../s3/Russian_River/imgOrig/IMG_4273.jpg
ensure_img_sanity ../s3/Russian_River/imgOrig/IMG_4298.jpg
ensure_img_sanity ../s3/Russian_River/imgOrig/IMG_4271.jpg
ensure_img_sanity ../s3/Russian_River/imgOrig/intermittent stream at the hairpin turn 1.jpg
ensure_img_sanity ../s3/Russian_River/imgOrig/IMG_4477.jpg
ensure_img_sanity ../s3/Russian_River/imgOrig/IMG_4305.jpg
ensure_img_sanity ../s3/Russian_River/imgOrig/IMG_4463.jpg
ensure_img_sanity ../s3/Russian_River/imgOr

...

processing place: ../s3/Costa_Rica_2018/
ensure_img_sanity ../s3/Costa_Rica_2018/imgOrig/PlayaTamborTurtleRelease1.jpg
ensure_img_sanity ../s3/Costa_Rica_2018/imgOrig/GuardiansOfCuru.jpg
ensure_img_sanity ../s3/Costa_Rica_2018/imgOrig/HotelLindaVista1.jpg
ensure_img_sanity ../s3/Costa_Rica_2018/imgOrig/Plant.jpg
ensure_img_sanity ../s3/Costa_Rica_2018/imgOrig/CuruRodentia.jpg
ensure_img_sanity ../s3/Costa_Rica_2018/imgOrig/PlayaTamborTurtleRelease0.jpg
ensure_img_sanity ../s3/Costa_Rica_2018/imgOrig/TamborTurtleRescueRioPanica.jpg
ensure_img_sanity ../s3/Costa_Rica_2018/imgOrig/TheMontezumaHatchery.jpg
ensure_img_sanity ../s3/Costa_Rica_2018/imgOrig/Ferry.jpg
ensure_img_sanity ../s3/Costa_Rica_2018/imgOrig/WesHangingBridges.jpg
ensure_img_sanity ../s3/Costa_Rica_2018/imgOrig/SavegreWaterfallHikeLimit.jpg
ensure_img_sanity ../s3/Costa_Rica_2018/imgOrig/MellissaMacaw.jpg
ensure_img_sanity ../s3/Costa_Rica_2018/imgOrig/SavegreWaterfall.jpg
ensure_img_sanity ../s3/Costa_Rica_2018/imgOrig/H

processing place: ../s3/Yucatan_2019/
ensure_img_sanity ../s3/Yucatan_2019/imgOrig/IMG_20191023_124129.jpg
ensure_img_sanity ../s3/Yucatan_2019/imgOrig/IMG_20191018_085944.jpg
ensure_img_sanity ../s3/Yucatan_2019/imgOrig/IMG_20191014_143950.jpg
ensure_img_sanity ../s3/Yucatan_2019/imgOrig/IMG_20191017_173309.jpg
ensure_img_sanity ../s3/Yucatan_2019/imgOrig/PANO_20191019_134619.vr.jpg
ensure_img_sanity ../s3/Yucatan_2019/imgOrig/IMG_20191020_161730.jpg
ensure_img_sanity ../s3/Yucatan_2019/imgOrig/IMG_20191021_112413.jpg
ensure_img_sanity ../s3/Yucatan_2019/imgOrig/IMG_20191019_085524.jpg
ensure_img_sanity ../s3/Yucatan_2019/imgOrig/IMG_20191022_085429.jpg
ensure_img_sanity ../s3/Yucatan_2019/imgOrig/IMG_20191020_121935.jpg
ensure_img_sanity ../s3/Yucatan_2019/imgOrig/IMG_20191015_085039.jpg
ensure_img_sanity ../s3/Yucatan_2019/imgOrig/IMG_20191019_130217.jpg
ensure_img_sanity ../s3/Yucatan_2019/imgOrig/IMG_20191022_122318.jpg
ensure_img_sanity ../s3/Yucatan_2019/imgOrig/IMG_20191025_104

processing place: ../s3/The_West/
ensure_img_sanity ../s3/The_West/imgOrig/A03D4CF2-80E3-40F4-8CD8-991B773B9535_1_105_c.jpeg


processing place: ../s3/Eel_River/
ensure_img_sanity ../s3/Eel_River/imgOrig/2017-07-03 08.38.48 HDR.jpg
ensure_img_sanity ../s3/Eel_River/imgOrig/2017-07-02 13.59.15 HDR.jpg
ensure_img_sanity ../s3/Eel_River/imgOrig/2017-07-01 17.52.01.jpg
ensure_img_sanity ../s3/Eel_River/imgOrig/2017-08-13 08.03.10 HDR.jpg
ensure_img_sanity ../s3/Eel_River/imgOrig/2017-07-02 09.21.40.jpg
ensure_img_sanity ../s3/Eel_River/imgOrig/2017-07-02 11.44.38 HDR.jpg
ensure_img_sanity ../s3/Eel_River/imgOrig/2017-08-13 15.51.16.jpg
ensure_img_sanity ../s3/Eel_River/imgOrig/2017-08-12 14.42.50 HDR.jpg
ensure_img_sanity ../s3/Eel_River/imgOrig/2017-07-03 07.10.52 HDR.jpg
ensure_img_sanity ../s3/Eel_River/imgOrig/2017-07-03 11.05.09.jpg
ensure_img_sanity ../s3/Eel_River/imgOrig/2017-07-02 09.39.14 HDR.jpg
ensure_img_sanity ../s3/Eel_River/imgOrig/2017-07-03 13.38.12 HDR.jpg
ensure_img_sanity ../s3/Eel_River/imgOrig/IMG_20190217_153745.jpg
ensure_img_sanity ../s3/Eel_River/imgOrig/2017-07-02 08.45.54 HDR.jpg
ensur

done.
